# Student Information
Name: Abner Tellez Sazo

Student ID: 106065434

---

### Instructions

- Download the dataset provided in this [link](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences#). The sentiment dataset contains a `sentence` and `score` label. Read what the dataset is about on the link provided before you start exploring it. 


- Then, you are asked to apply each of the data exploration and data operation techniques learned in the [first lab session](https://goo.gl/Sg4FS1) on the new dataset. You don't need to explain all the procedures as we did in the notebook, but you are expected to provide some **minimal comments** explaining your code. You are also expected to use the same libraries used in the first lab session. You are allowed to use and modify the `helper` functions we provided in the first lab session or create your own. Also, be aware that the helper functions may need modification as you are dealing with a completely different dataset. This part is worth 80% of your grade!


- After you have completed the operations, you should attempt the **bonus exercises** provided in the [notebook](https://goo.gl/Sg4FS1) we used for the first lab session. There are six (6) additional exercises; attempt them all, as it is part of your grade (10%). 


- You are also expected to tidy up your notebook and attempt new data operations that you have learned so far in the Data Mining course. Surprise us! This segment is worth 10% of your grade.


- After completing all the above tasks, you are free to remove this header block and submit your assignment following the guide provided in the `README.md` file of the assignment's [repository](https://github.com/omarsar/data_mining_hw_1). 

In [1]:
# necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import nltk
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import plotly.plotly as py
import plotly.graph_objs as go
import math
%matplotlib inline

# TA helpers
import helpers.data_mining_helpers as dmh
import helpers.text_analysis as ta

# My helpers
import helpers.csv_constructor as cc

#URL Data set
DATA_SET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"


# 1. Data source and 2. data preparation

In [3]:
# Load and prepare data directly the dataset URL
cc.download_zip(DATA_SET_URL) 

Extracting files...
Creating csv...
Done!


In [4]:
# Negative = 0, Positive = 1
categories = [0,1]

# 3. Converting dataset into Pandas dataframe

In [5]:
# Obtain the data trail using csv file in "dataset" folder
X = pd.read_csv("dataset/dataset.csv",delimiter="|")

In [6]:
# Show data_train and data_train size
print(X)

            Source                                           Sentence  Score
0     amazon_cells  So there is no way for me to plug it in here i...      0
1     amazon_cells                        Good case, Excellent value.      1
2     amazon_cells                             Great for the jawbone.      1
3     amazon_cells  Tied to charger for conversations lasting more...      0
4     amazon_cells                                  The mic is great.      1
5     amazon_cells  I have to jiggle the plug to get it to line up...      0
6     amazon_cells  If you have several dozen or several hundred c...      0
7     amazon_cells        If you are Razr owner...you must have this!      1
8     amazon_cells                Needless to say, I wasted my money.      0
9     amazon_cells                   What a waste of money and time!.      0
10    amazon_cells                    And the sound quality is great.      1
11    amazon_cells  He was very impressed when going from the orig...      1

In [7]:
# Print first 10 rows in data train
X[0:10]

,Source,Sentence,Score
0,amazon_cells,So there is no way for me to plug it in here i...,0
1,amazon_cells,"Good case, Excellent value.",1
2,amazon_cells,Great for the jawbone.,1
3,amazon_cells,Tied to charger for conversations lasting more...,0
4,amazon_cells,The mic is great.,1
5,amazon_cells,I have to jiggle the plug to get it to line up...,0
6,amazon_cells,If you have several dozen or several hundred c...,0
7,amazon_cells,If you are Razr owner...you must have this!,1
8,amazon_cells,"Needless to say, I wasted my money.",0
9,amazon_cells,What a waste of money and time!.,0


### 3.1 Try some queries 

In [8]:
# First 10 only source and score
X[0:10][["Source", "Score"]]

,Source,Score
0,amazon_cells,0
1,amazon_cells,1
2,amazon_cells,1
3,amazon_cells,0
4,amazon_cells,1
5,amazon_cells,0
6,amazon_cells,0
7,amazon_cells,1
8,amazon_cells,0
9,amazon_cells,0


In [9]:
# Last 10 all information
X[-11:-1]

,Source,Sentence,Score
2989,yelp,I would avoid this place if you are staying in...,0
2990,yelp,The refried beans that came with my meal were ...,0
2991,yelp,Spend your money and time some place else.,0
2992,yelp,A lady at the table next to us found a live gr...,0
2993,yelp,the presentation of the food was awful.,0
2994,yelp,I can't tell you how disappointed I was.,0
2995,yelp,I think food should have flavor and texture an...,0
2996,yelp,Appetite instantly gone.,0
2997,yelp,Overall I was not impressed and would not go b...,0
2998,yelp,"The whole experience was underwhelming, and I ...",0


In [10]:
# 30 lines in intervals of 100
X.iloc[::100, :][0:30]

,Source,Sentence,Score
0,amazon_cells,So there is no way for me to plug it in here i...,0
100,amazon_cells,Integrated seamlessly with the Motorola RAZR p...,1
200,amazon_cells,The color is even prettier than I thought it w...,1
300,amazon_cells,Sending it back.,0
400,amazon_cells,what a disappointment,0
500,amazon_cells,"The bose noise cancelling is amazing, which is...",1
600,amazon_cells,Their Research and Development division obviou...,1
700,amazon_cells,"Also, the phone doesn't seem to accept anythin...",0
800,amazon_cells,"Good , works fine.",1
900,amazon_cells,"This was utterly confusing at first, which cau...",0


## 4. Data Mining using Pandas

### 4.1 Missing Values

In [11]:
# Check if we have null values
X.isnull()

,Source,Sentence,Score
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [12]:
X.isnull().apply(lambda x: dmh.check_missing_values(x))

Source      (The amoung of missing records is: , 0)
Sentence    (The amoung of missing records is: , 0)
Score       (The amoung of missing records is: , 0)
dtype: object

**Add missing dummy data**

In [13]:
# Generate 300 rows of dummy data with missing values in our dataset
dummy_data = dmh.add_dummy_data({'Source': "dummy_site", 'Sentence': 'dummy_sentences', "Score" : 1}, 300, True)

In [14]:
dummy_data

[{'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Sentence': 'dummy_sentences'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_site'},
 {'Score': 1, 'Sentence': 'dummy_sentences'},
 {'Score': 1, 'Sentence': 'dummy_sentences'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_site'},
 {'Score': 1, 'Sentence': 'dummy_sentences'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Score': 1, 'Sentence': 'dummy_sentences'},
 {'Score': 1, 'Source': 'dummy_site'},
 {'Sentence': 'dummy_sentences', 'Source': 'dummy_s

In [15]:
X = X.append(dummy_data,ignore_index=True)

In [16]:
# check if the records was commited into result
len(X)

3300

In [17]:
X.isnull().apply(lambda x: dmh.check_missing_values(x))

Source      (The amoung of missing records is: , 105)
Sentence     (The amoung of missing records is: , 94)
Score       (The amoung of missing records is: , 101)
dtype: object

In [18]:
# Clear data of missing values
X.dropna(inplace=True)

In [19]:
X.isnull().apply(lambda x: dmh.check_missing_values(x))

Source      (The amoung of missing records is: , 0)
Sentence    (The amoung of missing records is: , 0)
Score       (The amoung of missing records is: , 0)
dtype: object

In [20]:
len(X)

3000

### 4.2 Check duplicate data

In [21]:
X.duplicated()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2970    False
2971    False
2972    False
2973    False
2974    False
2975    False
2976    False
2977    False
2978    False
2979    False
2980    False
2981    False
2982    False
2983    False
2984    False
2985    False
2986    False
2987    False
2988    False
2989    False
2990    False
2991    False
2992    False
2993    False
2994    False
2995    False
2996    False
2997    False
2998    False
2999    False
Length: 3000, dtype: bool

In [22]:
sum(X.duplicated())

17

In [23]:
# Show duplicates NEW QUERY :) 
X[X.duplicated(keep="first")]

,Source,Sentence,Score
285,amazon_cells,Great phone!.,1.0
407,amazon_cells,Works great.,1.0
524,amazon_cells,Works great!.,1.0
543,amazon_cells,Don't buy this product.,0.0
744,amazon_cells,If you like a loud buzzing to override all you...,0.0
748,amazon_cells,Does not fit.,0.0
778,amazon_cells,This is a great deal.,1.0
792,amazon_cells,Great Phone.,1.0
892,amazon_cells,Excellent product for the price.,1.0
896,amazon_cells,Great phone.,1.0


In [24]:
# Delete duplicates
X.drop_duplicates(keep="first", inplace=True)

In [25]:
sum(X.duplicated())

0

In [26]:
len(X)

2983

## 5. Data Processing

### 5.1 Sampling

In [27]:
X_sample = X.sample(n=1000)

In [28]:
len(X_sample)

1000

In [29]:
# 0 = Neagatives, 1 = Positives
categories

[0, 1]

In [30]:
X_category_counts = ta.get_tokens_and_frequency(list(X.Score))
X_sample_category_counts = ta.get_tokens_and_frequency(list(X_sample.Score))

In [31]:
py.iplot(ta.plot_word_frequency(X_category_counts, "Score distribution"))

In [32]:
py.iplot(ta.plot_word_frequency(X_sample_category_counts, "Sample score distribution"))

<hr>

# Exercise 1 ----> more kinds of represent the data using bar graphs! :)

In [33]:
X_amz = X_sample.loc[X_sample['Source'] == 'amazon_cells']
X_category_amz_counts = ta.get_tokens_and_frequency(list(X_amz.Score))

X_imdb = X_sample.loc[X_sample['Source'] == 'imdb']
X_category_imdb_counts = ta.get_tokens_and_frequency(list(X_imdb.Score))

X_yelp = X_sample.loc[X_sample['Source'] == 'yelp']
X_category_yelp_counts = ta.get_tokens_and_frequency(list(X_yelp.Score))

trace1 = go.Bar(
    x= list(X_category_amz_counts[0]),
    y= list(X_category_amz_counts[1]),
    name='Amazon Cells'
)
trace2 = go.Bar(
    x= list(X_category_imdb_counts[0]),
    y= list(X_category_imdb_counts[1]),
    name='IMDB'
)
trace3 = go.Bar(
    x= list(X_category_yelp_counts[0]),
    y= list(X_category_yelp_counts[1]),
    name='Yelp!'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

<hr>